In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from multicamera_acquisition.paths import DATA_DIR
from multicamera_acquisition.acquisition import acquire_video

/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [4]:
save_location = DATA_DIR / 'tests'

In [5]:
camera_list = [
    {'name': 'BackLeft', 'serial': 22181546, 'brand':'flir', 'gain': 12, "display":True},
    {'name': 'BackRight', 'serial': 22181539, 'brand':'flir', 'gain': 12, "display":True},
    {'name': 'FrontRight', 'serial': 22181541, 'brand':'flir', 'gain': 12, "display":True},
    {'name': 'FrontLeft', 'serial': 22181612, 'brand':'flir', 'gain': 12, "display":True},
    {'name': 'Bottom', 'serial': 22181550, 'brand':'flir', 'gain': 12, "display":True},
    {'name': 'Top', 'serial': 22181547, 'brand':'flir', 'gain': 12, "display":True},
]

In [ ]:
acquire_video(
    save_location,
    camera_list,
    framerate = 30,
    exposure_time = 2000,
    recording_duration_s = 60,
    serial_timeout_duration_s=0.1,
    overwrite=False,
    append_datetime=True,
)

INFO:root:Checking cameras...
INFO:root:Initializing cameras...
INFO:root:Camera BackLeft...
INFO:root:Initialized BackLeft (22181546)
INFO:root:Camera BackRight...
INFO:root:Initialized BackRight (22181539)
INFO:root:Camera FrontRight...
INFO:root:Initialized FrontRight (22181541)
INFO:root:Camera FrontLeft...
INFO:root:Initialized FrontLeft (22181612)
INFO:root:Camera Bottom...
INFO:root:Initialized Bottom (22181550)
INFO:root:Camera Top...
INFO:root:Initialized Top (22181547)
INFO:root:Preparing acquisition loops
INFO:root:Initializing Arduino...
INFO:root:BackLeft: Timeout occurred 
INFO:root:BackRight: Timeout occurred 
INFO:root:FrontRight: Timeout occurred 
INFO:root:FrontLeft: Timeout occurred 
INFO:root:Bottom: Timeout occurred 
INFO:root:Top: Timeout occurred 
INFO:root:BackLeft: Timeout occurred 


/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 0 frame on #22181546: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 0 frame on #22181539: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 0 frame on #22181541: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 0 frame on #22181612: 
SpinnakerException
  warnings.warn(


INFO:root:BackRight: Timeout occurred 


/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 0 frame on #22181550: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 0 frame on #22181547: 
SpinnakerException
  warnings.warn(


INFO:root:Starting Acquisition...
Confirmation recieved: Start


...

Finished
Finished
End Acquisition
INFO:root:Closing
INFO:root:FrontRight: Timeout occurred 


INFO:root:FrontLeft: Timeout occurred 
INFO:root:Bottom: Timeout occurred 
INFO:root:Top: Timeout occurred 
INFO:root:BackLeft: Timeout occurred 
INFO:root:BackRight: Timeout occurred 
INFO:root:FrontRight: Timeout occurred 
INFO:root:FrontLeft: Timeout occurred 
INFO:root:Bottom: Timeout occurred 


/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1801 frame on #22181612: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1801 frame on #22181550: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1801 frame on #22181541: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1801 frame on #22181546: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1801 frame on #22181547: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1801 frame on #22181539: 
SpinnakerE

INFO:root:Top: Timeout occurred 


/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1802 frame on #22181612: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1802 frame on #22181539: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1802 frame on #22181550: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1802 frame on #22181541: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1802 frame on #22181547: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:105: UserWarning: Dropped 1803 frame on #22181612: 
SpinnakerE